In [37]:
! pip install pandas

<h1>PART-1</h1>

In [38]:
import pandas as pd
import numpy as np 

rng = np.random.default_rng(seed = 0)

attendance_raw = [] 

cohort = ["alpha", "beta", "gamma"]

for i in range(1,25):
    attendance_dict = {}
    if i < 10:
        attendance_dict["student_id"] = f"S00{i}"
    else:
        attendance_dict["student_id"] = f"S0{i}"

    attendance_dict["cohort"] = rng.choice(cohort)

    attendance_dict["attended_sessions"] = rng.integers(0,7)
    
    attendance_dict["expected_sessions"] = 6
    
    attendance_raw.append(attendance_dict)

attendance = pd.DataFrame(attendance_raw)

print(attendance.head())
attendance.info()


  student_id cohort  attended_sessions  expected_sessions
0       S001  gamma                  4                  6
1       S002   beta                  1                  6
2       S003  alpha                  0                  6
3       S004  alpha                  0                  6
4       S005  alpha                  5                  6
<class 'pandas.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   student_id         24 non-null     str  
 1   cohort             24 non-null     str  
 2   attended_sessions  24 non-null     int64
 3   expected_sessions  24 non-null     int64
dtypes: int64(2), str(2)
memory usage: 900.0 bytes


In [39]:

# attendance.reset_index(inplace=True) i used this because if student_id  becomes index then it is removed from columns that is why i used reset
attendance_indexed = attendance.set_index("student_id")
attendance_indexed.head()


,cohort,attended_sessions,expected_sessions
student_id,,,
S001,gamma,4,6
S002,beta,1,6
S003,alpha,0,6
S004,alpha,0,6
S005,alpha,5,6


In [40]:
dict_id = {}

for i in range(1,11):
    j = rng.integers(1,35)
    if j < 10:
        a = str(f"S00{j}")
        if a not in dict_id:
            dict_id[a] = 1
        else:
            dict_id[a] += 1
    else:
        b = str(f"S0{j}")
        if b not in dict_id:
            dict_id[b] = 1
        else:
            dict_id[b] += 1


excused_absences = pd.Series(dict_id)

# print(excused_absences)

attendance_indexed["adjusted_attendance"] = attendance_indexed["attended_sessions"] + excused_absences
attendance_indexed["adjusted_attendance"] = attendance_indexed["adjusted_attendance"].fillna(attendance_indexed["attended_sessions"])
attendance_indexed

,cohort,attended_sessions,expected_sessions,adjusted_attendance
student_id,,,,
S001,gamma,4,6,4.0
S002,beta,1,6,1.0
S003,alpha,0,6,0.0
S004,alpha,0,6,0.0
S005,alpha,5,6,5.0
S006,beta,6,6,6.0
S007,beta,4,6,4.0
S008,gamma,5,6,5.0
S009,beta,3,6,4.0


<h1>PART-3</h1>

In [41]:
attendance["cohort"] = attendance["cohort"].replace({
    "alpha": "ALPHA ",
    "beta": "         bEta",
    "gamma": "     Gamma     "
})

attendance["cohort"] = attendance["cohort"].str.strip()
attendance["cohort"] = attendance["cohort"].str.lower()

attendance["cohort"].unique()

<StringArray>
['gamma', 'beta', 'alpha']
Length: 3, dtype: str

<h1>PART-4</h1>

In [42]:
low_attendance = attendance[attendance["attended_sessions"] < attendance["expected_sessions"]]


attendance

summary = attendance.groupby("cohort")["attended_sessions"].mean()

print(summary)

 
set(summary.index.unique()) == set(attendance["cohort"].unique())
# summary.reset_index("cohort",inplace=True)

cohort
alpha    2.777778
beta     3.200000
gamma    3.000000
Name: attended_sessions, dtype: float64


True

<h1>PART-5</h1>

In [43]:
attendance["attendance_ok"] = attendance["attended_sessions"] >= attendance["expected_sessions"]

low_attendance = attendance[attendance["attended_sessions"] < attendance["expected_sessions"]]

attendance

all(low_attendance["attendance_ok"] == False)
# low_attendance

True